In [72]:
import pandas as pd

In [73]:
# Ich will gucken welche Züge von Dortmund nach Düsseldorf am wenigsten verspätung haben
# 1. Alle Züge welche von Dortmund nach Düsseldorf fahren
# 2. groupby 


In [99]:
df = pd.read_csv("data.csv", converters={
    'arrival_time_delta': pd.to_timedelta,
    'departure_time_delta': pd.to_timedelta
})
#df.head()

In [98]:
#print(df["train_type"].unique())
df_local = df[df["train_type"].isin(["RE", "RB"])]
train_in_dortmund = df_local[df_local["station"] == "Dortmund Hbf"]
train_in_duesseldorf = df_local[df_local["station"] == "Düsseldorf Hbf"]

# Merge/join the two DataFrames on 'train_line_id' to compare 'train_line_station_num'
merged_df = train_in_duesseldorf.merge(
    train_in_dortmund[['train_line_id', 'train_line_station_num']],
    on='train_line_id',
    suffixes=('_duesseldorf', '_dortmund')
)

# Filter the merged DataFrame based on the comparison of 'train_line_station_num'
filtered_df = merged_df[merged_df['train_line_station_num_duesseldorf'] > merged_df['train_line_station_num_dortmund']]

mean_arrival_time_delta = filtered_df.groupby("train_name")["arrival_time_delta"].median()
mean_arrival_time_delta_minutes = mean_arrival_time_delta / pd.Timedelta(minutes=1)
mean_arrival_time_delta_minutes.reset_index()

,train_name,arrival_time_delta
0,RE 1,14.0
1,RE 4,4.5
2,RE 6,17.5
